<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/[NOTEBOOK_ID])

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community

Transform your AI ideas into reality through hands-on projects and expert mentorship.

[Start Your Journey](https://www.buildfastwithai.com/genai-course)

---

# Gemini 3 Pro - Specialized Use Cases

**Created by:** @BuildFastWithAI  
**Model:** Google Gemini 3 Pro  
**Last Updated:** November 2025

Advanced production use cases and integration patterns.

In [ ]:
!pip install -q google-generativeai langchain langchain-google-genai fastapi pydantic

In [ ]:
import google.generativeai as genai
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
import json

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## 1. Code Repository Analysis

In [ ]:
class CodeAnalyzer:
    def __init__(self):
        self.llm = ChatGoogleGenerativeAI(model="gemini-3-pro", temperature=0.3)
    
    def analyze_code(self, code: str, language: str = "python") -> dict:
        """Analyze code for quality and issues."""
        prompt = f"""
Analyze this {language} code and provide:
1. Code quality score (1-10)
2. Potential bugs
3. Performance issues
4. Security concerns
5. Suggestions for improvement

Code:
```{language}
{code}
```

Return as JSON.
"""
        
        response = self.llm.predict(prompt)
        try:
            return json.loads(response)
        except:
            return {"analysis": response}
    
    def generate_tests(self, code: str, language: str = "python") -> str:
        """Generate unit tests for code."""
        prompt = f"""
Generate comprehensive unit tests for this {language} code:

```{language}
{code}
```

Include:
- Normal cases
- Edge cases
- Error cases
"""
        
        return self.llm.predict(prompt)
    
    def document_code(self, code: str, language: str = "python") -> str:
        """Generate documentation for code."""
        prompt = f"""
Generate detailed documentation for this {language} code:

```{language}
{code}
```

Include:
- Function descriptions
- Parameter explanations
- Return value descriptions
- Usage examples
"""
        
        return self.llm.predict(prompt)

# Test
analyzer = CodeAnalyzer()

sample_code = """
def calculate_average(numbers):
    total = sum(numbers)
    return total / len(numbers)
"""

print("=== Code Analysis ===")
analysis = analyzer.analyze_code(sample_code)
print(json.dumps(analysis, indent=2))

print("\n=== Generated Tests ===")
tests = analyzer.generate_tests(sample_code)
print(tests[:300] + "...")

## 2. Production API Integration

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
import uvicorn

# Define models
class ChatRequest(BaseModel):
    message: str
    temperature: Optional[float] = 0.7
    max_tokens: Optional[int] = 1024

class ChatResponse(BaseModel):
    response: str
    model: str
    tokens_used: int

# Create API
app = FastAPI(title="Gemini 3 Pro API")

@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    """Chat endpoint."""
    try:
        model = genai.GenerativeModel(
            'gemini-3-pro',
            generation_config={
                "temperature": request.temperature,
                "max_output_tokens": request.max_tokens
            }
        )
        
        response = model.generate_content(request.message)
        
        return ChatResponse(
            response=response.text,
            model="gemini-3-pro",
            tokens_used=len(request.message.split()) + len(response.text.split())
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health():
    return {"status": "healthy", "model": "gemini-3-pro"}

# To run: uvicorn main:app --reload
print("API defined. In production, run with: uvicorn main:app --host 0.0.0.0 --port 8000")

## 3. Advanced Evaluation & Benchmarking

In [ ]:
import time
from typing import List, Dict

class ModelEvaluator:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.llm = ChatGoogleGenerativeAI(model="gemini-3-pro")
    
    def evaluate_accuracy(self, qa_pairs: List[Dict]) -> dict:
        """Evaluate model accuracy on Q&A pairs."""
        results = []
        
        for pair in qa_pairs:
            question = pair["question"]
            expected = pair["expected_answer"]
            
            response = self.model.generate_content(question)
            actual = response.text
            
            # Evaluate similarity
            eval_prompt = f"""
Rate how similar these answers are (0-10):

Expected: {expected}
Actual: {actual}

Return only the number.
"""
            
            score = float(self.llm.predict(eval_prompt).strip())
            results.append({
                "question": question,
                "score": score,
                "actual": actual[:100]
            })
        
        avg_score = sum(r["score"] for r in results) / len(results)
        
        return {
            "average_score": avg_score,
            "results": results
        }
    
    def benchmark_performance(self, prompts: List[str]) -> dict:
        """Benchmark model performance."""
        times = []
        token_counts = []
        
        for prompt in prompts:
            start = time.time()
            response = self.model.generate_content(prompt)
            elapsed = time.time() - start
            
            times.append(elapsed)
            token_counts.append(len(prompt.split()) + len(response.text.split()))
        
        return {
            "avg_latency": sum(times) / len(times),
            "min_latency": min(times),
            "max_latency": max(times),
            "avg_tokens": sum(token_counts) / len(token_counts),
            "throughput": len(prompts) / sum(times)
        }

# Test evaluation
evaluator = ModelEvaluator()

qa_pairs = [
    {
        "question": "What is 2+2?",
        "expected_answer": "4"
    },
    {
        "question": "What is the capital of France?",
        "expected_answer": "Paris"
    }
]

print("=== Accuracy Evaluation ===")
accuracy = evaluator.evaluate_accuracy(qa_pairs)
print(f"Average Score: {accuracy['average_score']:.2f}/10")

print("\n=== Performance Benchmark ===")
benchmark = evaluator.benchmark_performance([
    "Explain AI",
    "What is ML?",
    "Define quantum computing"
])
print(json.dumps(benchmark, indent=2))

## 4. Monitoring & Analytics

In [ ]:
from datetime import datetime
from collections import defaultdict

class ModelMonitor:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.metrics = defaultdict(list)
    
    def tracked_generate(self, prompt: str, **kwargs) -> dict:
        """Generate content with tracking."""
        start = time.time()
        
        try:
            response = self.model.generate_content(prompt, **kwargs)
            elapsed = time.time() - start
            
            metrics = {
                "timestamp": datetime.now().isoformat(),
                "prompt_length": len(prompt),
                "response_length": len(response.text),
                "latency": elapsed,
                "status": "success",
                "error": None
            }
            
            result = {"text": response.text, "metrics": metrics}
            
        except Exception as e:
            elapsed = time.time() - start
            metrics = {
                "timestamp": datetime.now().isoformat(),
                "prompt_length": len(prompt),
                "latency": elapsed,
                "status": "error",
                "error": str(e)
            }
            result = {"text": None, "metrics": metrics}
        
        # Store metrics
        for key, value in metrics.items():
            self.metrics[key].append(value)
        
        return result
    
    def get_stats(self) -> dict:
        """Get aggregated statistics."""
        latencies = [m for m in self.metrics["latency"] if isinstance(m, (int, float))]
        
        return {
            "total_requests": len(self.metrics["timestamp"]),
            "success_rate": sum(1 for s in self.metrics["status"] if s == "success") / len(self.metrics["status"]),
            "avg_latency": sum(latencies) / len(latencies) if latencies else 0,
            "p95_latency": sorted(latencies)[int(len(latencies) * 0.95)] if latencies else 0,
            "avg_prompt_length": sum(self.metrics["prompt_length"]) / len(self.metrics["prompt_length"]),
            "errors": [e for e in self.metrics["error"] if e is not None]
        }

# Test monitoring
monitor = ModelMonitor()

# Make some requests
for prompt in ["What is AI?", "Explain ML", "Define data science"]:
    result = monitor.tracked_generate(prompt)
    print(f"Response: {result['text'][:100]}...")

# Get stats
print("\n=== Monitoring Stats ===")
stats = monitor.get_stats()
print(json.dumps(stats, indent=2))

## 5. Cost Optimization

In [ ]:
class CostOptimizer:
    def __init__(self, cost_per_1k_tokens: float = 0.001):
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.cost_per_1k = cost_per_1k_tokens
        self.cache = {}
    
    def estimate_cost(self, prompt: str, response: str = None) -> float:
        """Estimate request cost."""
        prompt_tokens = len(prompt.split())
        response_tokens = len(response.split()) if response else 100  # Estimate
        total_tokens = prompt_tokens + response_tokens
        return (total_tokens / 1000) * self.cost_per_1k
    
    def optimize_prompt(self, prompt: str) -> str:
        """Optimize prompt for cost."""
        # Remove unnecessary whitespace
        optimized = " ".join(prompt.split())
        
        # Estimate savings
        original_cost = self.estimate_cost(prompt)
        optimized_cost = self.estimate_cost(optimized)
        savings = original_cost - optimized_cost
        
        print(f"Original cost: ${original_cost:.6f}")
        print(f"Optimized cost: ${optimized_cost:.6f}")
        print(f"Savings: ${savings:.6f}")
        
        return optimized
    
    def cached_generate(self, prompt: str) -> dict:
        """Generate with caching to reduce costs."""
        if prompt in self.cache:
            return {
                "text": self.cache[prompt],
                "cached": True,
                "cost": 0
            }
        
        response = self.model.generate_content(prompt)
        self.cache[prompt] = response.text
        
        return {
            "text": response.text,
            "cached": False,
            "cost": self.estimate_cost(prompt, response.text)
        }

# Test optimization
optimizer = CostOptimizer()

prompt = """What    is    artificial    intelligence?    
Please    explain    in    detail."""

print("=== Prompt Optimization ===")
optimized = optimizer.optimize_prompt(prompt)

print("\n=== Caching Benefit ===")
result1 = optimizer.cached_generate("What is AI?")
print(f"First call - Cost: ${result1['cost']:.6f}")

result2 = optimizer.cached_generate("What is AI?")
print(f"Second call (cached) - Cost: ${result2['cost']:.6f}")

## Key Takeaways

✅ **Production Patterns:**
- Code analysis and generation
- API integration with FastAPI
- Comprehensive evaluation
- Real-time monitoring
- Cost optimization

📌 **Best Practices:**
- Implement caching
- Monitor performance metrics
- Optimize prompts for cost
- Add comprehensive error handling
- Use structured logging

🚀 **Deployment Checklist:**
- [ ] API rate limiting
- [ ] Cost monitoring
- [ ] Error tracking
- [ ] Performance metrics
- [ ] Security measures
- [ ] Caching strategy
- [ ] Load testing

🔗 **Resources:**
- [Production Best Practices](https://ai.google.dev/)
- Follow [@BuildFastWithAI](https://twitter.com/BuildFastWithAI)